<a href="https://colab.research.google.com/github/saitejakarre/Saiteja/blob/main/NER_end_to_end_code.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np

In [ ]:
from datasets import load_dataset
from transformers import AutoTokenizer, AutoModelForTokenClassification, DataCollatorForTokenClassification, TrainingArguments, Trainer
import numpy as np

In [ ]:
pip install "datasets<4.0.0"

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 491.5/491.5 kB 32.0 MB/s eta 0:00:00
  Attempting uninstall: datasets
    Found existing installation: datasets 4.0.0
    Uninstalling datasets-4.0.0:
      Successfully uninstalled datasets-4.0.0


In [ ]:
dataset = load_dataset("conll2003",trust_remote_code=True)

README.md: 0.00B [00:00, ?B/s]

conll2003.py: 0.00B [00:00, ?B/s]

Generating train split:   0%|          | 0/14041 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/3250 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/3453 [00:00<?, ? examples/s]

In [ ]:
dataset

DatasetDict({
    train: Dataset({
        features: ['id', 'tokens', 'pos_tags', 'chunk_tags', 'ner_tags'],
        num_rows: 14041
    })
    validation: Dataset({
        features: ['id', 'tokens', 'pos_tags', 'chunk_tags', 'ner_tags'],
        num_rows: 3250
    })
    test: Dataset({
        features: ['id', 'tokens', 'pos_tags', 'chunk_tags', 'ner_tags'],
        num_rows: 3453
    })
})

In [ ]:
label_list = dataset["train"].features["ner_tags"].feature.names
num_labels = len(label_list)

In [ ]:
model_checkpoint = "distilbert-base-uncased"
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)

tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/483 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

In [ ]:
model = AutoModelForTokenClassification.from_pretrained(
    model_checkpoint,
    num_labels=num_labels
)

model.safetensors:   0%|          | 0.00/268M [00:00<?, ?B/s]

Some weights of DistilBertForTokenClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
def tokenize_and_align_labels(examples):
    tokenized_inputs = tokenizer(
        examples["tokens"], truncation=True, is_split_into_words=True
    )
    labels = []
    for i, label in enumerate(examples["ner_tags"]):
        word_ids = tokenized_inputs.word_ids(batch_index=i)
        label_ids = []
        previous_word_id = None
        for word_id in word_ids:
            if word_id is None:

                label_ids.append(-100)
            elif word_id != previous_word_id:
                label_ids.append(label[word_id])
            else:
                label_ids.append(-100)
            previous_word_id = word_id
        labels.append(label_ids)
    tokenized_inputs["labels"] = labels
    return tokenized_inputs


In [ ]:
tokenized_datasets = dataset.map(tokenize_and_align_labels, batched=True)


Map:   0%|          | 0/14041 [00:00<?, ? examples/s]

Map:   0%|          | 0/3250 [00:00<?, ? examples/s]

Map:   0%|          | 0/3453 [00:00<?, ? examples/s]

In [ ]:
data_collator = DataCollatorForTokenClassification(tokenizer)

In [ ]:
!pip install "evaluate==0.4.0"

import evaluate

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 81.4/81.4 kB 8.5 MB/s eta 0:00:00


In [ ]:
!pip install seqeval

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.6/43.6 kB 3.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for seqeval: filename=seqeval-1.2.2-py3-none-any.whl size=16162 sha256=0d7fc1fa6efa3aa9a474f3a62a60fffce7e2c6a43bd0c7e33b6983e807ada88c
  Stored in directory: /root/.cache/pip/wheels/bc/92/f0/243288f899c2eacdfa8c5f9aede4c71a9bad0ee26a01dc5ead
Successfully built seqeval


In [ ]:
metric = evaluate.load("seqeval")

In [ ]:
def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    predictions = np.argmax(predictions, axis=2)

    true_labels = [[label_list[l] for l in label if l != -100] for label in labels]
    true_predictions = [
        [label_list[p] for (p, l) in zip(prediction, label) if l != -100]
        for prediction, label in zip(predictions, labels)
    ]
    results = metric.compute(predictions=true_predictions, references=true_labels)
    return {
        "precision": results["overall_precision"],
        "recall": results["overall_recall"],
        "f1": results["overall_f1"],
        "accuracy": results["overall_accuracy"],
    }

In [ ]:
training_args = TrainingArguments(
    output_dir="./ner-distilbert",
    eval_strategy="epoch",
    save_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=3,
    weight_decay=0.01,
    logging_dir="./logs",
    report_to="none",
    logging_steps=10
)

In [ ]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["validation"],
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics
)



/tmp/ipython-input-4055128089.py:1: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


In [ ]:
trainer.train()

Epoch,Training Loss,Validation Loss,Precision,Recall,F1,Accuracy
1,0.076600,0.054632,0.912048,0.924941,0.918449,0.984366
2,0.023200,0.051294,0.924107,0.940592,0.932277,0.986585
3,0.019400,0.052588,0.930488,0.939414,0.934930,0.986702


TrainOutput(global_step=2634, training_loss=0.06870215584155548, metrics={'train_runtime': 257.0631, 'train_samples_per_second': 163.862, 'train_steps_per_second': 10.247, 'total_flos': 510122266253334.0, 'train_loss': 0.06870215584155548, 'epoch': 3.0})

In [ ]:
trainer.save_model("./ner-distilbert")

In [ ]:
from transformers import pipeline

In [ ]:
ner_pipeline = pipeline(
    "ner",
    model="./ner-distilbert",  # path to your saved model
    tokenizer="./ner-distilbert",
    aggregation_strategy="simple"  # combines subword tokens into whole entities
)


Device set to use cuda:0


In [ ]:
def decode_labels(results):
    decoded = []
    for ent in results:
        label_id = int(ent["entity_group"].split("_")[-1])  # e.g., "LABEL_3" → 3
        ent["entity_group"] = label_list[label_id]          # map to actual label
        decoded.append(ent)
    return decoded


In [ ]:
sentence = "SaiTeja living in Tirumala with lord venkateshwara."
raw_results = ner_pipeline(sentence)



In [ ]:
final_results = decode_labels(raw_results)

In [ ]:
for entity in final_results:
    print(f"Entity: {entity['word']}, Label: {entity['entity_group']}, Score: {entity['score']:.4f}")


Entity: sai, Label: B-PER, Score: 0.9766
Entity: ##teja, Label: I-PER, Score: 0.6450
Entity: living in, Label: O, Score: 0.9994
Entity: tirum, Label: B-LOC, Score: 0.8647
Entity: ##ala, Label: I-LOC, Score: 0.9700
Entity: with lord, Label: O, Score: 0.9965
Entity: ve, Label: B-PER, Score: 0.9919
Entity: ##nkateshwara, Label: I-PER, Score: 0.7125
Entity: ., Label: O, Score: 0.9995
